### 优化神经网络的手段
1. 激活函数的优化
2. 学习率采用指数衰减学习率
3. 参数的更新使用滑动平均值
4. 防止过拟合加入正则化项

#### 激活函数

根据实际情况选择合适的激活函数， 加入激活函数是为了解决线性模型不能解决异或问题，因为引入了非线性，提高了模型的表达力，使模型具有更好的区分度；<br>损失函数(loss function) : 预测值(y) 与已知答案(y_) 的差距； NN优化目标：loss最小；<br>
主流的loss 计算有三种：均方误差MSE、交叉熵CE(cross entropy)、自定义

#### MSE 激活函数

In [ ]:
loss_mse = reduce_mean(tf.square(y - y_))

#### 交叉熵激活函数
交叉熵ce(cross entropy): 表征两个概率分布之间的距离，交叉熵越大，两个概率分布越远， 交叉熵越小，两个概率分布越近<br>
后面的参数 1e-12 和 1.0 都是对y做限制，为了使输入的y有意义， 当y小于1e-12时，就让y=1e-12, 当y大于1.0的时候，就让y=1.0

In [ ]:
ce = - tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-12, 1.0)))

在实际操作中，为了让前向传播计算出的结果满足概率分布，也就是让推测出的n分类的n个输出(y1, y2, …, yn), 
每一个都在[0, 1] 之间，并且这n个输出的和为1. 
通过softmax()函数实现, y1, y2 … yn 这n个数通过softmax函数处理后，得到的结果是符合概率分布的。
将输入y1, y2, … yn 通过softmax函数后，再与标准答案y,求交叉熵

In [ ]:
ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
cem = tf.reduce_mean(ce)

#### 学习率

学习率：就是每次参数更新的幅度；参数的更新，向着损失函数梯度下降的方向； 目的是找到某个w, 使得损失函数的梯度最小，也就是loss的导数为0；优化参数的目关于学习率设置的问题：
<br>
关于学习率设置需要合适：如果学习率过大，会导致震荡而不收敛; 如果学习率过小，会导致收敛速度极慢
<br>
指数衰减的学习率：先下降的速度比较快，之后缓慢的下降<br>
学习率基数(初始值)* 学习率衰减率(0~1)^ 多少轮更新一次学习率 = 总样本数 / batch_size

In [ ]:
learning_rate = LEARNING_RATE_BASE * LEARNING_RATE_DECAY^(global_step / LEARNING_RATE_STEP)

# 计数器，记录当前共运行了多少轮，只用于计数，所以我们将属性标注为不可训练
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, global_step, LEARNING_RATE_STEP, LEARNING_RATE_DECAY, staircase=True)

"""
LEARNING_RATE_BASE: 学习率的基数，最开始设置的学习率
LEARNING_RATE_DECAY: 学习率衰减率
staircase=True: 表示呈阶梯曲线衰减
staircase=True: 表示学习率是平滑下降的曲线
"""

# 设置损失函数 loss=(w+1)^2, 令w初值是常数10， 反向传播就是求最优值w,即求最小loss对应的w值
# 使用指数衰减的学习率，在迭代初期得到较高的下降速度，可以在较小的训练轮数下取得更有收敛度

LEARNING_RATE_BASE = 0.1 # 最初学习率
LEARNING_RATE_DECAY = 0.99 # 学习率衰减
LEARNING_RATE_STEP = 1  # 喂入多少轮batch_size后，更新一次学习率，一般设置为: 总样本数 / batch_size

# 运行了几轮BATCH_SIZE的计数器，初始值为0， 设为不被训练
global_step = tf.Variable(0, trainable=False)

# 定义指数下降学习率,在梯度下降中，学习率不再是固定的值，而是动态调整的
learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, global_step, LEARNING_RATE_STEP, LEARNING_RATE_DECAY, staircase=True)

# 定义待优化参数，初始值为5
w = tf.Variable(tf.constant(5, dytype=tf.float32))

# 定义损失函数loss
loss = tf.square(w+1)

# 定义反向传播方法,这里的学习率learning_rate不再是一个定值了，而是动态的变化的；
# 有一点不太明白的是 global_step=global_step 是什么意思？
# global_step refers to the number of batches seen by the graph. 
# Every time a batch is provided, the weights are updated in the direction that minimizes the loss. 
# global_step just keeps track of the number of batches seen so far. 
# When it is passed in the minimize() argument list, the variable is increased by one.

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

# 生成会话，训练40轮
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    for i in range(40):
        sess.run(train_step)
        # 保存学习率的值，用于打印输出
        learning_rate_val = sess.run(learning_rate)
        global_step_val = sess.run(global_step)
        w_val = sess.run(w)
        loss_val = sess.run(loss)
        print("After %s steps， global_step is %f, w is %f, learning rate is %f, loss is %f" %(i, global_step_val, w_val, learning_rate_val, loss_val))
  

In [ ]:
"""
滑动平均 ema
滑动平均(影子值)：记录了每一个参数一段时间内过往值的平均
不仅表示了当前值，还表示了过去一段时间内的平均值，这样可以增加模型的泛化性
针对所有参数：w 和 b 进行滑动平均
滑动平均就像是给参数加了影子，参数变化，影子缓慢跟随

影子初始值 = 参数初始值
影子 = 衰减率 * 影子 + (1 - 衰减率) * 参数
衰减率 = min{MOVING_AVERAGE_DECAY, (1+轮数)/(10+轮数)}

MOVING_AVERAGE_DECAY 是一个超参数，一般赋一个比较大的值，比如0.99， 参数w1为0， 轮数global_step为0， w1的滑动平均值为0
当参数w1被更新为1的时候： 得到w1的滑动平均值
"""

In [ ]:
# MOVING_AVERAGE_DECAY 滑动平均衰减率
# global_step 当前轮数

ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)

# 定义了对括号里的参数求滑动平均
ema_op = ema.apply([])

# 自动将所有待训练的参数汇总成列表
ema_op = ema.apply(tf.trainable_varibales())

# 每运行此句，所有待优化的参数求滑动平均
# 将计算互动平均和训练过程绑定在一起运行，合成一个训练节点
with tf.control_dependencies([train_step, ema_op]):
  train_op = tf.no_op(name="train")

# 查看某个参数的滑动平均值
ema.avarage(参数名)


#1 定义变量以及滑动平均类
# 定义一个32位浮点变量，初始值为0.0, 这个代码就是不断更新w1参数，优化w1参数， 滑动平均做了个w1的影子
w1 = tf.Variable(0, dtype=tf.float32)

# 定义num_updates(NN的迭代次数), 初始值为0， 不可被优化(训练),这个参数不训练
golbal_step = tf.Variable(0, trainable=False)

# 实例化滑动平均类，衰减率为0.99， 当前轮数：global_step
MOVING_AVERAGE_DECAY = 0.99
ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)

# ema.apply后面的括号里是更新列表，每次运行sess.run(ema_op)时， 对更新列表中的元素求滑动平均
# 在实际应用中会使用tf.trainable_variables()自动将所有待训练的参数汇总为列表
# ema.apply([w1])
ema_op = ema.apply(tf.trainable_variables())

#2 查看不同迭代中变量取值的变化
with tf.Session() as sess:
    # 初始化
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    # 用ema.average(w1)获取w1滑动平均(要运行多个节点， 作为列表中的元素列出，写在sess.run中)
    # 打印出当前参数w1和w1的滑动平均
    print(sess.run([w1, ema.average(w1)]))
  
    # 参数w1的值赋值为1
    sess.run(tf.assign(w1, 1))
    sess.run(rma_op)
    print(sess.run([w1, ema.average(w1)]))
  
    # 更新step和w1的值，模拟出100轮迭代后，参数w1变为10
    sess.run(tf.assign(global_step, 100))
    sess.run(tf.assign(w1, 10))
    sess.run(rma_op)
    print(sess.run([w1, ema.average(w1)]))
  
    # 每次sess.run会更新一次w1的滑动平均值
    sess.run(ema_op)
    print(sess.run([w1, ema.average(w1)]))

In [ ]:
"""
正则化 regularization
包含了正则化的模型曲线会更加的平滑，数据集中的噪声将会更小
正则化缓解过拟合
正则化在损失函数中引入模型复杂度指标，利用给W加权值，弱化了训练数据的噪声(一般不正则化b)
loss = loss(y, y_) + REGULARIZER \* loss(w)
loss(y, y_) : 模型中所有参数的损失函数，如：交叉熵、均方误差mse
REGULARIZER : 用超参数REGULARIZER给出参数w在总loss中的比例，即正则化的权重
"""

In [ ]:
# L1正则化,对参数的绝对值求和
loss(w) = tf.contrib.layers.l1_regularizer(REGULARIZER)(w)

# L2正则化,对参数的平方求和
loss(w) = tf.contrib.layers.l2_regularizer(REGULARIZER)(w)

# 把计算好了的所有w正则化加入到集合losses中做加法
tf.add_to_collection("losses", tf.contrib.layers.l2_regularizer(regularizer)(w))
# tf.add_n 可以将losses中的所有值相加； 交叉熵 + 正则项 = loss 函数
loss = cem + tf.add_n(tf.get_collection("losses"))

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

BATCH_SIZE = 30
seed = 2

# 基于seed产生随机数
rdm = np.random.RandomState(seed)

# 随机数返回300行2列的矩阵中取出一行，判断如果两个坐标的平方和小雨2， 给y赋值1，其余赋值0，作为标签
Y_ = [int(x0*x0 + x1*x1 < 2) for (x0, x1) in X ]

# 遍历Y中的每个元素，1 赋值为'red', 其余赋值为‘'blue', 便于可视化显示
Y_c = [['red' if y else 'blur'] for y in Y_]
  
# 对数据集X和标签Y进行形状整理，第一个元素为-1表示根据最后列计算，第二个元素表示多少列，可见X为2列，Y为1列
# X 是n行两列，Y 是n行1列
X = np.vstack(X).reshape(-1, 2)
Y_ = np.vstack(Y_).reshape(-1, 1)

# 用plt.scatter 画出离散点
plt.scatter(X[:,0], X[:,1], c=np.squeeze(Y_c))
plt.show()

# 定义神经网络的输入、参数和输出，定义前向传播过程
def get_weight(shape, regularizer):
    w = tf.Variable(tf.random_normal(shape), dtype=tf.float32)
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(regularizer)(w))
    return w

def get_bias(shape):
    b = tf.Variable(tf.constant(0.01, shape=shape))
    return b

# 占位
x = tf.placeholder(tf.float32, shape=(None, 2))
y_ = tf.placeholder(tf.float32, shape=(None, 1))

# 输入层2个节点，隐藏层11个节点，输出层1个节点
w1 = get_weight([2, 11], 0.01)
# 偏置 b1是11个
b1 = get_bias([11])
y1 = tf.nn.relu(tf.matmul(x, w1) + b1)	# 要过激活函数

w2 = get_weight([11, 1], 0.01)
b2 = get_bias([1])
y = tf.matmul(y1, w2) + b2		# 输出层不过激活

# 定义损失函数为mean square error
loss_mse = tf.reduce_mean(tf.square(y - y_))
loss_total = loss_mse + tf.add_n(tf.get_collection('losses'))

# 定义反向传播方法：如果不含正则化
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss_mse)
# 定义反向传播方法：如果包含正则化
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss_total)

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    # 训练40000次，每次喂入BATCH_SIZE个数据
    STEPS = 40000
    for i in range(STEPS):
        start = (i*BATCH_SIZE) % 300
        end = start + BATCH_SIZE
        sess.run(train_step, feed_dict={x:X[start:end], y_:Y_[start:end]})
    if i%2000 == 0:
        loss_mse_v = sess.run(loss_mse, feed_dict={x:X, y_:Y_})
        print("After %d steps, loss is: %f" %(i, loss_v))
  
    # xx在-3到3之间以步长0.01为间距生成网格坐标，组成xx,yy坐标集，二维坐标点
    xx, yy = np.mgrid[-3:3:0.01, -3:3:0.01]
    
    # 将xx,yy拉直，并合并成一个2列的矩阵，得到一个网格坐标点的集合
    grid = np.c_[xx.ravel(), yy.ravel()]
    
    # 将网格坐标点喂入神经网络，probs为输出，也就是神经网络推算出的结果
    probs = sess.run(y, feed_dict={x:grid})
    
    # probs的shape调整成xx的样子
    probs = probs.reshape(xx.shape)

plt.scatter(X[:, 0], X[:, 1], c=np.squeeze(Y_c))
# 画等高线,为probs为0.5的点上色
plt.contour(xx, yy, probs, levels=[.5])
plt.show()